# Richmeister Integration Test Setup

This book allows the installation of test infrastructure, including tables in two regions and installation and configuration of the replication functions for multi-master replication of the tables.

## Asset Deployment

In [ ]:
# Deploy to the east region
import os
deploy_east = os.environ['DEPLOY_BUCKET_EAST']

In [ ]:
%%script env DEPLOY_BUCKET="$deploy_east" bash
cd ../s2fq/
make package
make deploy

In [ ]:
# Deploy to the west
deploy_west = os.environ['DEPLOY_BUCKET_WEST']

In [ ]:
%%script env DEPLOY_BUCKET="$deploy_west" bash
cd ../s2fq/
make package
make deploy

## Test Table Creation

In [ ]:
import boto3

ddb_east = boto3.client('dynamodb', region_name='us-east-1')
ddb_west = boto3.client('dynamodb', region_name='us-west-2')

table_name = 'TestTable'

In [ ]:
create_table_east_response = ddb_east.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    },
    StreamSpecification={
        'StreamEnabled': True,
        'StreamViewType': 'NEW_AND_OLD_IMAGES'
    }
)

print create_table_east_response

In [ ]:
table_east_stream = create_table_east_response['TableDescription']['LatestStreamArn']
print table_east_stream

In [ ]:
create_table_west_response = ddb_west.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'Id',
            'AttributeType': 'N'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'Id',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 1,
        'WriteCapacityUnits': 1
    },
    StreamSpecification={
        'StreamEnabled': True,
        'StreamViewType': 'NEW_AND_OLD_IMAGES'
    }
)

print create_table_west_response

In [ ]:
table_west_stream = create_table_west_response['TableDescription']['LatestStreamArn']
print table_west_stream

## Install Stream to Queue Stack

In [ ]:
cf_east = boto3.client('cloudformation', region_name='us-east-1')
cf_west = boto3.client('cloudformation', region_name='us-west-2')

s2q_east_stack = 's2qEast'
s2q_west_stack = 's2qWest'

In [ ]:
# East
template_url = 'https://s3.amazonaws.com/' + deploy_east + '/s2fq.yml'

stack_east = cf_east.create_stack(
    StackName=s2q_east_stack,
    TemplateURL=template_url,
    Parameters=[
        {
            'ParameterKey': 'TableName',
            'ParameterValue': table_name
        },
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': deploy_east
        },
        {
            'ParameterKey': 'StreamArn',
            'ParameterValue': table_east_stream
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print stack_east

In [ ]:
# West
template_url = 'https://s3-us-west-2.amazonaws.com/' + deploy_west + '/s2fq.yml'

stack_west = cf_west.create_stack(
    StackName=s2q_west_stack,
    TemplateURL=template_url,
    Parameters=[
        {
            'ParameterKey': 'TableName',
            'ParameterValue': table_name
        },
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': deploy_west
        },
        {
            'ParameterKey': 'StreamArn',
            'ParameterValue': table_west_stream
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print stack_west

In [ ]:
# Wait for stack creation to complete

print 'waiting for east stack complete...'
waiter = cf_east.get_waiter('stack_create_complete')
waiter.wait(
    StackName=s2q_east_stack
)

print 'waiting for west stack complete...'
waiter = cf_west.get_waiter('stack_create_complete')
waiter.wait(
    StackName=s2q_west_stack
)

## Clean Up

In [ ]:
print 'destroy {}'.format(s2q_east_stack)
response = cf_east.delete_stack(
    StackName=s2q_east_stack
)

print 'destroy {}'.format(s2q_west_stack)
response = cf_west.delete_stack(
    StackName=s2q_west_stack
)

print 'waiting for east stack delete...'
waiter = cf_east.get_waiter('stack_delete_complete')
waiter.wait(
    StackName=s2q_east_stack
)

print 'waiting for west stack delete...'
waiter = cf_west.get_waiter('stack_delete_complete')
waiter.wait(
    StackName=s2q_west_stack
)

In [ ]:
# Delete test tables
for client in [ddb_east, ddb_west]:
    response = client.delete_table(
        TableName = table_name
    )
    
    print response